In [3]:
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
#import statsmodels.graphics.api as smg
#import patsy
#get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from scipy import stats
#import seaborn as sns
amazon = pd.read_csv('/home/w/Data Mining/HW1/amazon/train.csv')
data =amazon
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [4]:
data.describe()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000
mean,0.942110,42923.916171,25988.957979,116952.627788,118301.823156,118912.779914,125916.152644,170178.369648,183703.408893,119789.430132
std,0.233539,34173.892702,35928.031650,10875.563591,4551.588572,18961.322917,31036.465825,69509.462130,100488.407413,5784.275516
min,0.000000,0.000000,25.000000,4292.000000,23779.000000,4674.000000,117879.000000,4673.000000,3130.000000,117880.000000
25%,1.000000,20299.000000,4566.000000,117961.000000,118102.000000,118395.000000,118274.000000,117906.000000,118363.000000,118232.000000
50%,1.000000,35376.000000,13545.000000,117961.000000,118300.000000,118921.000000,118568.000000,128696.000000,119006.000000,118570.000000
75%,1.000000,74189.000000,42034.000000,117961.000000,118386.000000,120535.000000,120006.000000,235280.000000,290919.000000,119348.000000
max,1.000000,312153.000000,311696.000000,311178.000000,286791.000000,286792.000000,311867.000000,311867.000000,308574.000000,270691.000000


In [17]:
# 由於ROLE_CODE和ROLE_FAMILY與ROLE_TITLE存在一對一和一對多的關係，認為他不能包含更多的信息，刪去這兩個變量
data = amazon
del data["ROLE_CODE"]
del data["ROLE_FAMILY"]

In [18]:
 #利用循環，得到每個自變量出現的頻率，賦值到新的列中。
one= ["RESOURCE","MGR_ID","ROLE_ROLLUP_1","ROLE_ROLLUP_2","ROLE_DEPTNAME","ROLE_TITLE","ROLE_FAMILY_DESC"]
for i in range(0,len(one)):
    a=data[one[i]]
    b=data[one[i]].value_counts()/32769
    a=a.map(b)
    data[one[i]+"_prob"]=a
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,ROLE_ROLLUP_1_prob,ROLE_ROLLUP_2_prob,ROLE_DEPTNAME_prob,ROLE_TITLE_prob,ROLE_FAMILY_DESC_prob
0,1,39353,85475,117961,118300,123472,117905,117906,0.000092,0.001678,0.653270,0.135006,0.002197,0.109341,0.210443
1,1,17183,1540,117961,118343,123125,118536,118536,0.000915,0.000305,0.653270,0.120388,0.004852,0.002472,0.000366
2,1,36724,14457,118219,118220,117884,117879,267952,0.000061,0.000092,0.005615,0.005615,0.016662,0.038329,0.001007
3,1,36135,5396,117961,118343,119993,118321,240983,0.000031,0.001892,0.653270,0.120388,0.005798,0.141872,0.037963
4,1,42680,5905,117929,117930,119569,119323,123932,0.000244,0.000275,0.008423,0.004211,0.001373,0.002289,0.000580


In [19]:
# 利用循環，得到每兩個自變量同時出現的頻率，賦值到新的列中。
two = ["RESOURCE","MGR_ID","ROLE_ROLLUP_1","ROLE_ROLLUP_2","ROLE_DEPTNAME","ROLE_TITLE","ROLE_FAMILY_DESC"] 
for i in range(0,len(two)):
    for j in range(i+1,len(two)):
        a=data[two[i]]+data[two[j]]*1000000
        b=a.value_counts()/32769
        a=a.map(b)
        data[two[i]+"_"+two[j]+"_prob"]=a
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,...,ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob,ROLE_ROLLUP_1_ROLE_DEPTNAME_prob,ROLE_ROLLUP_1_ROLE_TITLE_prob,ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob,ROLE_ROLLUP_2_ROLE_DEPTNAME_prob,ROLE_ROLLUP_2_ROLE_TITLE_prob,ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob,ROLE_DEPTNAME_ROLE_TITLE_prob,ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob,ROLE_TITLE_ROLE_FAMILY_DESC_prob
0,1,39353,85475,117961,118300,123472,117905,117906,0.000092,0.001678,...,0.135006,0.002014,0.089200,0.180659,0.002014,0.013855,0.033233,0.000671,0.001678,0.079557
1,1,17183,1540,117961,118343,123125,118536,118536,0.000915,0.000305,...,0.120388,0.003815,0.002472,0.000366,0.003754,0.000580,0.000153,0.000153,0.000153,0.000366
2,1,36724,14457,118219,118220,117884,117879,267952,0.000061,0.000092,...,0.005615,0.000397,0.001556,0.000061,0.000397,0.001556,0.000061,0.005615,0.000061,0.000061
3,1,36135,5396,117961,118343,119993,118321,240983,0.000031,0.001892,...,0.120388,0.005401,0.125057,0.036956,0.005035,0.022460,0.007782,0.003052,0.001770,0.016204
4,1,42680,5905,117929,117930,119569,119323,123932,0.000244,0.000275,...,0.004211,0.000671,0.000488,0.000305,0.000549,0.000244,0.000244,0.000183,0.000183,0.000519


In [20]:
# 利用循環，得到每三個自變量同時出現的頻率，賦值到新的列中。
three = ["RESOURCE","MGR_ID","ROLE_ROLLUP_1","ROLE_ROLLUP_2","ROLE_DEPTNAME","ROLE_TITLE","ROLE_FAMILY_DESC"] 
for i in range(0,len(three)):
     for j in range(i+1,len(three)):
            for k in range(j+1,len(three)):
                a = data[three[i]]*100000*100000+data[three[j]]*1000000+data[three[k]]
                b = a.value_counts()/91690
                a = a.map(b)
                data[three[i]+"_"+three[j]+"_"+three[k]+"_"+"prob"]=a
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,...,ROLE_ROLLUP_1_ROLE_ROLLUP_2_ROLE_DEPTNAME_prob,ROLE_ROLLUP_1_ROLE_ROLLUP_2_ROLE_TITLE_prob,ROLE_ROLLUP_1_ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob,ROLE_ROLLUP_1_ROLE_DEPTNAME_ROLE_TITLE_prob,ROLE_ROLLUP_1_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob,ROLE_ROLLUP_1_ROLE_TITLE_ROLE_FAMILY_DESC_prob,ROLE_ROLLUP_2_ROLE_DEPTNAME_ROLE_TITLE_prob,ROLE_ROLLUP_2_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob,ROLE_ROLLUP_2_ROLE_TITLE_ROLE_FAMILY_DESC_prob,ROLE_DEPTNAME_ROLE_TITLE_ROLE_FAMILY_DESC_prob
0,1,39353,85475,117961,118300,123472,117905,117906,0.000092,0.001678,...,0.000720,0.004951,0.011877,0.000185,0.000556,0.023220,0.000185,0.000556,0.003937,0.000218
1,1,17183,1540,117961,118343,123125,118536,118536,0.000915,0.000305,...,0.001341,0.000207,0.000055,0.000055,0.000055,0.000131,0.000055,0.000055,0.000055,0.000055
2,1,36724,14457,118219,118220,117884,117879,267952,0.000061,0.000092,...,0.000142,0.000556,0.000022,0.000055,0.000022,0.000022,0.000055,0.000022,0.000022,0.000022
3,1,36135,5396,117961,118343,119993,118321,240983,0.000031,0.001892,...,0.001800,0.008027,0.002781,0.001091,0.000633,0.005682,0.000971,0.000534,0.001451,0.000545
4,1,42680,5905,117929,117930,119569,119323,123932,0.000244,0.000275,...,0.000196,0.000087,0.000087,0.000044,0.000044,0.000109,0.000022,0.000022,0.000087,0.000065


In [21]:
# 利用循環，得到每三個自變量和RESOURCE同時出現的頻率，賦值到新的列中。
four = ["RESOURCE","MGR_ID","ROLE_ROLLUP_1","ROLE_ROLLUP_2","ROLE_DEPTNAME","ROLE_TITLE","ROLE_FAMILY_DESC"] 
for i in range(1,len(four)):
     for j in range(i+1,len(four)):
            for k in  range(j+1,len(four)):
                a = data[four[0]]*100000*100000+data[four[i]]*1000000+data[four[j]]+data[four[k]]*0.000001
                b = a.value_counts()/32769
                a = a.map(b)
                data[four[0]+"_"+four[i]+"_"+four[j]+"_"+four[k]+"_"+"prob"]=a
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,...,RESOURCE_ROLE_ROLLUP_1_ROLE_ROLLUP_2_ROLE_DEPTNAME_prob,RESOURCE_ROLE_ROLLUP_1_ROLE_ROLLUP_2_ROLE_TITLE_prob,RESOURCE_ROLE_ROLLUP_1_ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob,RESOURCE_ROLE_ROLLUP_1_ROLE_DEPTNAME_ROLE_TITLE_prob,RESOURCE_ROLE_ROLLUP_1_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob,RESOURCE_ROLE_ROLLUP_1_ROLE_TITLE_ROLE_FAMILY_DESC_prob,RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_ROLE_TITLE_prob,RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob,RESOURCE_ROLE_ROLLUP_2_ROLE_TITLE_ROLE_FAMILY_DESC_prob,RESOURCE_ROLE_DEPTNAME_ROLE_TITLE_ROLE_FAMILY_DESC_prob
0,1,39353,85475,117961,118300,123472,117905,117906,0.000092,0.001678,...,0.000092,0.000092,0.000092,0.000031,0.000031,0.000061,0.000031,0.000031,0.000061,0.000031
1,1,17183,1540,117961,118343,123125,118536,118536,0.000915,0.000305,...,0.000336,0.000305,0.000153,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031
2,1,36724,14457,118219,118220,117884,117879,267952,0.000061,0.000092,...,0.000061,0.000061,0.000031,0.000061,0.000031,0.000031,0.000061,0.000031,0.000031,0.000031
3,1,36135,5396,117961,118343,119993,118321,240983,0.000031,0.001892,...,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031
4,1,42680,5905,117929,117930,119569,119323,123932,0.000244,0.000275,...,0.000061,0.000061,0.000061,0.000061,0.000061,0.000092,0.000031,0.000031,0.000061,0.000061


In [22]:
# RESOURCE 確定時其他單個變量同時發生的概率
resourcetwo = ['RESOURCE_MGR_ID_prob','RESOURCE_ROLE_ROLLUP_1_prob', 'RESOURCE_ROLE_ROLLUP_2_prob','RESOURCE_ROLE_DEPTNAME_prob', 
               'RESOURCE_ROLE_TITLE_prob','RESOURCE_ROLE_FAMILY_DESC_prob']
for i in range(0,len(resourcetwo)):
    a =  data[resourcetwo[i]]/data.RESOURCE_prob
    data[resourcetwo[i]+"_"+"probre"]=a
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,...,RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_ROLE_TITLE_prob,RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob,RESOURCE_ROLE_ROLLUP_2_ROLE_TITLE_ROLE_FAMILY_DESC_prob,RESOURCE_ROLE_DEPTNAME_ROLE_TITLE_ROLE_FAMILY_DESC_prob,RESOURCE_MGR_ID_prob_probre,RESOURCE_ROLE_ROLLUP_1_prob_probre,RESOURCE_ROLE_ROLLUP_2_prob_probre,RESOURCE_ROLE_DEPTNAME_prob_probre,RESOURCE_ROLE_TITLE_prob_probre,RESOURCE_ROLE_FAMILY_DESC_prob_probre
0,1,39353,85475,117961,118300,123472,117905,117906,0.000092,0.001678,...,0.000031,0.000031,0.000061,0.000031,1.000000,1.000000,1.000000,0.333333,0.666667,1.000000
1,1,17183,1540,117961,118343,123125,118536,118536,0.000915,0.000305,...,0.000031,0.000031,0.000031,0.000031,0.033333,0.866667,0.366667,0.033333,0.033333,0.033333
2,1,36724,14457,118219,118220,117884,117879,267952,0.000061,0.000092,...,0.000061,0.000031,0.000031,0.000031,0.500000,1.000000,1.000000,1.000000,1.000000,0.500000
3,1,36135,5396,117961,118343,119993,118321,240983,0.000031,0.001892,...,0.000031,0.000031,0.000031,0.000031,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,1,42680,5905,117929,117930,119569,119323,123932,0.000244,0.000275,...,0.000031,0.000031,0.000061,0.000061,0.250000,0.375000,0.250000,0.250000,0.500000,0.375000


In [23]:
# 其他單個變量確定時RESOURCE變量同時發生的概率
resourcetwo = ['RESOURCE_MGR_ID_prob','RESOURCE_ROLE_ROLLUP_1_prob', 'RESOURCE_ROLE_ROLLUP_2_prob','RESOURCE_ROLE_DEPTNAME_prob', 
               'RESOURCE_ROLE_TITLE_prob','RESOURCE_ROLE_FAMILY_DESC_prob']
resourceone = [ 'MGR_ID_prob', 'ROLE_ROLLUP_1_prob','ROLE_ROLLUP_2_prob', 'ROLE_DEPTNAME_prob', 'ROLE_TITLE_prob','ROLE_FAMILY_DESC_prob']
for i in range(0,len(resourcetwo)):
    a =  data[resourcetwo[i]]/data[resourceone[i]]
    data[resourcetwo[i]+"_"+"proboth"]=a
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,...,RESOURCE_ROLE_ROLLUP_2_prob_probre,RESOURCE_ROLE_DEPTNAME_prob_probre,RESOURCE_ROLE_TITLE_prob_probre,RESOURCE_ROLE_FAMILY_DESC_prob_probre,RESOURCE_MGR_ID_prob_proboth,RESOURCE_ROLE_ROLLUP_1_prob_proboth,RESOURCE_ROLE_ROLLUP_2_prob_proboth,RESOURCE_ROLE_DEPTNAME_prob_proboth,RESOURCE_ROLE_TITLE_prob_proboth,RESOURCE_ROLE_FAMILY_DESC_prob_proboth
0,1,39353,85475,117961,118300,123472,117905,117906,0.000092,0.001678,...,1.000000,0.333333,0.666667,1.000000,0.054545,0.000140,0.000678,0.013889,0.000558,0.000435
1,1,17183,1540,117961,118343,123125,118536,118536,0.000915,0.000305,...,0.366667,0.033333,0.033333,0.033333,0.100000,0.001215,0.002788,0.006289,0.012346,0.083333
2,1,36724,14457,118219,118220,117884,117879,267952,0.000061,0.000092,...,1.000000,1.000000,1.000000,0.500000,0.333333,0.010870,0.010870,0.003663,0.001592,0.030303
3,1,36135,5396,117961,118343,119993,118321,240983,0.000031,0.001892,...,1.000000,1.000000,1.000000,1.000000,0.016129,0.000047,0.000253,0.005263,0.000215,0.000804
4,1,42680,5905,117929,117930,119569,119323,123932,0.000244,0.000275,...,0.250000,0.250000,0.500000,0.375000,0.222222,0.010870,0.014493,0.044444,0.053333,0.157895


In [24]:
# RESOURCE 確定時其他兩個變量同時發生的概率
resourcethree = [ 'RESOURCE_MGR_ID_ROLE_ROLLUP_1_prob','RESOURCE_MGR_ID_ROLE_ROLLUP_2_prob', 'RESOURCE_MGR_ID_ROLE_DEPTNAME_prob',
        'RESOURCE_MGR_ID_ROLE_TITLE_prob','RESOURCE_MGR_ID_ROLE_FAMILY_DESC_prob', 'RESOURCE_ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob',
        'RESOURCE_ROLE_ROLLUP_1_ROLE_DEPTNAME_prob','RESOURCE_ROLE_ROLLUP_1_ROLE_TITLE_prob','RESOURCE_ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob',
        'RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_prob','RESOURCE_ROLE_ROLLUP_2_ROLE_TITLE_prob', 'RESOURCE_ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob',
        'RESOURCE_ROLE_DEPTNAME_ROLE_TITLE_prob', 'RESOURCE_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob','RESOURCE_ROLE_TITLE_ROLE_FAMILY_DESC_prob']
for i in range(0,len(resourcethree)):
    a =  data[resourcethree[i]]/data.RESOURCE_prob
    data[resourcethree[i]+"_"+"probre"]=a
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,...,RESOURCE_ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob_probre,RESOURCE_ROLE_ROLLUP_1_ROLE_DEPTNAME_prob_probre,RESOURCE_ROLE_ROLLUP_1_ROLE_TITLE_prob_probre,RESOURCE_ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob_probre,RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_prob_probre,RESOURCE_ROLE_ROLLUP_2_ROLE_TITLE_prob_probre,RESOURCE_ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob_probre,RESOURCE_ROLE_DEPTNAME_ROLE_TITLE_prob_probre,RESOURCE_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob_probre,RESOURCE_ROLE_TITLE_ROLE_FAMILY_DESC_prob_probre
0,1,39353,85475,117961,118300,123472,117905,117906,0.000092,0.001678,...,0.357389,0.119130,0.238259,0.357389,0.119130,0.238259,0.357389,0.119130,0.119130,0.238259
1,1,17183,1540,117961,118343,123125,118536,118536,0.000915,0.000305,...,0.131043,0.011913,0.011913,0.011913,0.011913,0.011913,0.011913,0.011913,0.011913,0.011913
2,1,36724,14457,118219,118220,117884,117879,267952,0.000061,0.000092,...,0.357389,0.357389,0.357389,0.178695,0.357389,0.357389,0.178695,0.357389,0.178695,0.178695
3,1,36135,5396,117961,118343,119993,118321,240983,0.000031,0.001892,...,0.357389,0.357389,0.357389,0.357389,0.357389,0.357389,0.357389,0.357389,0.357389,0.357389
4,1,42680,5905,117929,117930,119569,119323,123932,0.000244,0.000275,...,0.089347,0.089347,0.134021,0.134021,0.044674,0.089347,0.089347,0.089347,0.089347,0.134021


In [25]:
# 其他兩個變量確定時RESOURCE變量同時發生的概率
resourcethree = [ 'RESOURCE_MGR_ID_ROLE_ROLLUP_1_prob','RESOURCE_MGR_ID_ROLE_ROLLUP_2_prob', 'RESOURCE_MGR_ID_ROLE_DEPTNAME_prob',
        'RESOURCE_MGR_ID_ROLE_TITLE_prob','RESOURCE_MGR_ID_ROLE_FAMILY_DESC_prob', 'RESOURCE_ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob',
        'RESOURCE_ROLE_ROLLUP_1_ROLE_DEPTNAME_prob','RESOURCE_ROLE_ROLLUP_1_ROLE_TITLE_prob','RESOURCE_ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob',
        'RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_prob','RESOURCE_ROLE_ROLLUP_2_ROLE_TITLE_prob', 'RESOURCE_ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob',
        'RESOURCE_ROLE_DEPTNAME_ROLE_TITLE_prob', 'RESOURCE_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob','RESOURCE_ROLE_TITLE_ROLE_FAMILY_DESC_prob']
othertwo = ['MGR_ID_ROLE_ROLLUP_1_prob','MGR_ID_ROLE_ROLLUP_2_prob','MGR_ID_ROLE_DEPTNAME_prob', 'MGR_ID_ROLE_TITLE_prob',
       'MGR_ID_ROLE_FAMILY_DESC_prob', 'ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob', 'ROLE_ROLLUP_1_ROLE_DEPTNAME_prob', 'ROLE_ROLLUP_1_ROLE_TITLE_prob',
       'ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob', 'ROLE_ROLLUP_2_ROLE_DEPTNAME_prob', 'ROLE_ROLLUP_2_ROLE_TITLE_prob',
       'ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob', 'ROLE_DEPTNAME_ROLE_TITLE_prob','ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob',
       'ROLE_TITLE_ROLE_FAMILY_DESC_prob']
for i in range(0,len(resourcethree)):
    a =  data[resourcethree[i]]/data[othertwo[i]]
    data[othertwo[i]+"_"+"proboth"]=a
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,...,ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob_proboth,ROLE_ROLLUP_1_ROLE_DEPTNAME_prob_proboth,ROLE_ROLLUP_1_ROLE_TITLE_prob_proboth,ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob_proboth,ROLE_ROLLUP_2_ROLE_DEPTNAME_prob_proboth,ROLE_ROLLUP_2_ROLE_TITLE_prob_proboth,ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob_proboth,ROLE_DEPTNAME_ROLE_TITLE_prob_proboth,ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob_proboth,ROLE_TITLE_ROLE_FAMILY_DESC_prob_proboth
0,1,39353,85475,117961,118300,123472,117905,117906,0.000092,0.001678,...,0.000242,0.005415,0.000245,0.000181,0.005415,0.001574,0.000985,0.016245,0.006498,0.000274
1,1,17183,1540,117961,118343,123125,118536,118536,0.000915,0.000305,...,0.000997,0.002859,0.004412,0.029782,0.002906,0.018810,0.071478,0.071478,0.071478,0.029782
2,1,36724,14457,118219,118220,117884,117879,267952,0.000061,0.000092,...,0.003885,0.054983,0.014015,0.178695,0.054983,0.014015,0.178695,0.003885,0.178695,0.178695
3,1,36135,5396,117961,118343,119993,118321,240983,0.000031,0.001892,...,0.000091,0.002019,0.000087,0.000295,0.002166,0.000486,0.001402,0.003574,0.006162,0.000673
4,1,42680,5905,117929,117930,119569,119323,123932,0.000244,0.000275,...,0.005180,0.032490,0.067010,0.107217,0.019855,0.089347,0.089347,0.119130,0.119130,0.063069


In [26]:
#build MODEL
# 劃分測試集與訓練集
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve,roc_auc_score,classification_report 
y = data.ACTION
X = data
del X["ACTION"]
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [38]:
# 利用以上處理所得的共133個自變量建立隨機森林模型
forest = RandomForestClassifier(criterion='gini', 
                             n_estimators=1000,
                             min_samples_split=12,
                             min_samples_leaf=1,
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
RFfit = forest.fit(X_train , y_train)

In [39]:
test = pd.read_csv('/home/w/Data Mining/HW1/amazon/test.csv')

In [40]:
# 由於ROLE_CODE和ROLE_FAMILY與ROLE_TITLE存在一對一和一對多的關係，認為他不能包含更多的信息，刪去這兩個變量
data = test
del data["ROLE_CODE"]
del data["ROLE_FAMILY"]
del data["id"]

 #利用循環，得到每個自變量出現的頻率，賦值到新的列中。
one= ["RESOURCE","MGR_ID","ROLE_ROLLUP_1","ROLE_ROLLUP_2","ROLE_DEPTNAME","ROLE_TITLE","ROLE_FAMILY_DESC"]
for i in range(0,len(one)):
    a=data[one[i]]
    b=data[one[i]].value_counts()/32769
    a=a.map(b)
    data[one[i]+"_prob"]=a
data.head()

# 利用循環，得到每兩個自變量同時出現的頻率，賦值到新的列中。
two = ["RESOURCE","MGR_ID","ROLE_ROLLUP_1","ROLE_ROLLUP_2","ROLE_DEPTNAME","ROLE_TITLE","ROLE_FAMILY_DESC"] 
for i in range(0,len(two)):
    for j in range(i+1,len(two)):
        a=data[two[i]]+data[two[j]]*1000000
        b=a.value_counts()/32769
        a=a.map(b)
        data[two[i]+"_"+two[j]+"_prob"]=a
data.head()

# 利用循環，得到每三個自變量同時出現的頻率，賦值到新的列中。
three = ["RESOURCE","MGR_ID","ROLE_ROLLUP_1","ROLE_ROLLUP_2","ROLE_DEPTNAME","ROLE_TITLE","ROLE_FAMILY_DESC"] 
for i in range(0,len(three)):
     for j in range(i+1,len(three)):
            for k in range(j+1,len(three)):
                a = data[three[i]]*100000*100000+data[three[j]]*1000000+data[three[k]]
                b = a.value_counts()/91690
                a = a.map(b)
                data[three[i]+"_"+three[j]+"_"+three[k]+"_"+"prob"]=a
data.head()

# 利用循環，得到每三個自變量和RESOURCE同時出現的頻率，賦值到新的列中。
four = ["RESOURCE","MGR_ID","ROLE_ROLLUP_1","ROLE_ROLLUP_2","ROLE_DEPTNAME","ROLE_TITLE","ROLE_FAMILY_DESC"] 
for i in range(1,len(four)):
     for j in range(i+1,len(four)):
            for k in  range(j+1,len(four)):
                a = data[four[0]]*100000*100000+data[four[i]]*1000000+data[four[j]]+data[four[k]]*0.000001
                b = a.value_counts()/32769
                a = a.map(b)
                data[four[0]+"_"+four[i]+"_"+four[j]+"_"+four[k]+"_"+"prob"]=a
data.head()

# RESOURCE 確定時其他單個變量同時發生的概率
resourcetwo = ['RESOURCE_MGR_ID_prob','RESOURCE_ROLE_ROLLUP_1_prob', 'RESOURCE_ROLE_ROLLUP_2_prob','RESOURCE_ROLE_DEPTNAME_prob', 
               'RESOURCE_ROLE_TITLE_prob','RESOURCE_ROLE_FAMILY_DESC_prob']
for i in range(0,len(resourcetwo)):
    a =  data[resourcetwo[i]]/data.RESOURCE_prob
    data[resourcetwo[i]+"_"+"probre"]=a
data.head()

# 其他單個變量確定時RESOURCE變量同時發生的概率
resourcetwo = ['RESOURCE_MGR_ID_prob','RESOURCE_ROLE_ROLLUP_1_prob', 'RESOURCE_ROLE_ROLLUP_2_prob','RESOURCE_ROLE_DEPTNAME_prob', 
               'RESOURCE_ROLE_TITLE_prob','RESOURCE_ROLE_FAMILY_DESC_prob']
resourceone = [ 'MGR_ID_prob', 'ROLE_ROLLUP_1_prob','ROLE_ROLLUP_2_prob', 'ROLE_DEPTNAME_prob', 'ROLE_TITLE_prob','ROLE_FAMILY_DESC_prob']
for i in range(0,len(resourcetwo)):
    a =  data[resourcetwo[i]]/data[resourceone[i]]
    data[resourcetwo[i]+"_"+"proboth"]=a
data.head()

# RESOURCE 確定時其他兩個變量同時發生的概率
resourcethree = [ 'RESOURCE_MGR_ID_ROLE_ROLLUP_1_prob','RESOURCE_MGR_ID_ROLE_ROLLUP_2_prob', 'RESOURCE_MGR_ID_ROLE_DEPTNAME_prob',
        'RESOURCE_MGR_ID_ROLE_TITLE_prob','RESOURCE_MGR_ID_ROLE_FAMILY_DESC_prob', 'RESOURCE_ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob',
        'RESOURCE_ROLE_ROLLUP_1_ROLE_DEPTNAME_prob','RESOURCE_ROLE_ROLLUP_1_ROLE_TITLE_prob','RESOURCE_ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob',
        'RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_prob','RESOURCE_ROLE_ROLLUP_2_ROLE_TITLE_prob', 'RESOURCE_ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob',
        'RESOURCE_ROLE_DEPTNAME_ROLE_TITLE_prob', 'RESOURCE_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob','RESOURCE_ROLE_TITLE_ROLE_FAMILY_DESC_prob']
for i in range(0,len(resourcethree)):
    a =  data[resourcethree[i]]/data.RESOURCE_prob
    data[resourcethree[i]+"_"+"probre"]=a
data.head()

# 其他兩個變量確定時RESOURCE變量同時發生的概率
resourcethree = [ 'RESOURCE_MGR_ID_ROLE_ROLLUP_1_prob','RESOURCE_MGR_ID_ROLE_ROLLUP_2_prob', 'RESOURCE_MGR_ID_ROLE_DEPTNAME_prob',
        'RESOURCE_MGR_ID_ROLE_TITLE_prob','RESOURCE_MGR_ID_ROLE_FAMILY_DESC_prob', 'RESOURCE_ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob',
        'RESOURCE_ROLE_ROLLUP_1_ROLE_DEPTNAME_prob','RESOURCE_ROLE_ROLLUP_1_ROLE_TITLE_prob','RESOURCE_ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob',
        'RESOURCE_ROLE_ROLLUP_2_ROLE_DEPTNAME_prob','RESOURCE_ROLE_ROLLUP_2_ROLE_TITLE_prob', 'RESOURCE_ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob',
        'RESOURCE_ROLE_DEPTNAME_ROLE_TITLE_prob', 'RESOURCE_ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob','RESOURCE_ROLE_TITLE_ROLE_FAMILY_DESC_prob']
othertwo = ['MGR_ID_ROLE_ROLLUP_1_prob','MGR_ID_ROLE_ROLLUP_2_prob','MGR_ID_ROLE_DEPTNAME_prob', 'MGR_ID_ROLE_TITLE_prob',
       'MGR_ID_ROLE_FAMILY_DESC_prob', 'ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob', 'ROLE_ROLLUP_1_ROLE_DEPTNAME_prob', 'ROLE_ROLLUP_1_ROLE_TITLE_prob',
       'ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob', 'ROLE_ROLLUP_2_ROLE_DEPTNAME_prob', 'ROLE_ROLLUP_2_ROLE_TITLE_prob',
       'ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob', 'ROLE_DEPTNAME_ROLE_TITLE_prob','ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob',
       'ROLE_TITLE_ROLE_FAMILY_DESC_prob']
for i in range(0,len(resourcethree)):
    a =  data[resourcethree[i]]/data[othertwo[i]]
    data[othertwo[i]+"_"+"proboth"]=a
data.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,ROLE_ROLLUP_1_prob,...,ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob_proboth,ROLE_ROLLUP_1_ROLE_DEPTNAME_prob_proboth,ROLE_ROLLUP_1_ROLE_TITLE_prob_proboth,ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob_proboth,ROLE_ROLLUP_2_ROLE_DEPTNAME_prob_proboth,ROLE_ROLLUP_2_ROLE_TITLE_prob_proboth,ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob_proboth,ROLE_DEPTNAME_ROLE_TITLE_prob_proboth,ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob_proboth,ROLE_TITLE_ROLE_FAMILY_DESC_prob_proboth
0,78766,72734,118079,118080,117878,117879,118177,0.001373,0.000366,0.016754,...,0.026690,0.026690,0.028099,0.023283,0.026690,0.028099,0.023283,0.010492,0.023283,0.020200
1,40644,4378,117961,118327,118507,118863,122008,0.000153,0.001251,1.149196,...,0.000073,0.000985,0.003885,0.007446,0.000985,0.003885,0.007446,0.003574,0.007446,0.007446
2,75443,2395,117961,118300,119488,118172,301534,0.000824,0.002563,1.149196,...,0.000925,0.009927,0.011529,0.001921,0.009927,0.011529,0.003340,0.017869,0.009659,0.011913
3,43219,19986,117961,118225,118403,120773,136187,0.000153,0.002197,1.149196,...,0.000403,0.001623,0.000921,0.013746,0.001623,0.004896,0.013746,0.012764,0.019855,0.008717
4,42093,50015,117961,118343,119598,118422,300136,0.013519,0.001007,1.149196,...,0.006968,0.011865,0.012176,0.012996,0.012047,0.012902,0.012918,0.012431,0.012013,0.014429


In [41]:
testset = data
testset.head(3)

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,RESOURCE_prob,MGR_ID_prob,ROLE_ROLLUP_1_prob,...,ROLE_ROLLUP_1_ROLE_ROLLUP_2_prob_proboth,ROLE_ROLLUP_1_ROLE_DEPTNAME_prob_proboth,ROLE_ROLLUP_1_ROLE_TITLE_prob_proboth,ROLE_ROLLUP_1_ROLE_FAMILY_DESC_prob_proboth,ROLE_ROLLUP_2_ROLE_DEPTNAME_prob_proboth,ROLE_ROLLUP_2_ROLE_TITLE_prob_proboth,ROLE_ROLLUP_2_ROLE_FAMILY_DESC_prob_proboth,ROLE_DEPTNAME_ROLE_TITLE_prob_proboth,ROLE_DEPTNAME_ROLE_FAMILY_DESC_prob_proboth,ROLE_TITLE_ROLE_FAMILY_DESC_prob_proboth
0,78766,72734,118079,118080,117878,117879,118177,0.001373,0.000366,0.016754,...,0.026690,0.026690,0.028099,0.023283,0.026690,0.028099,0.023283,0.010492,0.023283,0.020200
1,40644,4378,117961,118327,118507,118863,122008,0.000153,0.001251,1.149196,...,0.000073,0.000985,0.003885,0.007446,0.000985,0.003885,0.007446,0.003574,0.007446,0.007446
2,75443,2395,117961,118300,119488,118172,301534,0.000824,0.002563,1.149196,...,0.000925,0.009927,0.011529,0.001921,0.009927,0.011529,0.003340,0.017869,0.009659,0.011913


In [42]:
preds = RFfit.predict(testset)

In [43]:
submit = pd.read_csv('/home/w/Data Mining/HW1/amazon/sampleSubmission.csv')

submit['Action'] = preds

submit.to_csv('/home/w/Data Mining/HW1/amazon/cdn01.csv', index= False)

In [57]:
preds.show

AttributeError: 'numpy.ndarray' object has no attribute 'show'

In [3]:
from __future__ import division

import numpy as np
from sklearn import (metrics, cross_validation, linear_model, preprocessing)

SEED = 42  # always use a seed for randomized procedures


def load_data(filename, use_labels=True):
    """
    Load data from CSV files and return them as numpy arrays
    The use_labels parameter indicates whether one should
    read the first column (containing class labels). If false,
    return all 0s. 
    """

    # load column 1 to 8 (ignore last one)
    data = np.loadtxt(open(filename), delimiter=',',
                      usecols=range(1, 9), skiprows=1)
    if use_labels:
        labels = np.loadtxt(open(filename), delimiter=',',
                            usecols=[0], skiprows=1)
    else:
        labels = np.zeros(data.shape[0])
    return labels, data


def save_results(predictions, filename):
    """Given a vector of predictions, save results in CSV format."""
    with open(filename, 'w') as f:
        f.write("id,ACTION\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (i + 1, pred))


def main():
    """
    Fit models and make predictions.
    We'll use one-hot encoding to transform our categorical features
    into binary features.
    y and X will be numpy array objects.
    """
    model = linear_model.LogisticRegression(C=3)  # the classifier we'll use

    # === load data in memory === #
    print "loading data"
    y, X = load_data('/home/w/Data Mining/HW1/amazon/train.csv')
    y_test, X_test = load_data('/home/w/Data Mining/HW1/amazon/test.csv', use_labels=False)

    # === one-hot encoding === #
    # we want to encode the category IDs encountered both in
    # the training and the test set, so we fit the encoder on both
    encoder = preprocessing.OneHotEncoder()
    encoder.fit(np.vstack((X, X_test)))
    X = encoder.transform(X)  # Returns a sparse matrix (see numpy.sparse)
    X_test = encoder.transform(X_test)

    # if you want to create new features, you'll need to compute them
    # before the encoding, and append them to your dataset after

    # === training & metrics === #
    mean_auc = 0.0
    n = 10  # repeat the CV procedure 10 times to get more precise results
    for i in range(n):
        # for each iteration, randomly hold out 20% of the data as CV set
        X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(
            X, y, test_size=.20, random_state=i*SEED)

        # if you want to perform feature selection / hyperparameter
        # optimization, this is where you want to do it

        # train model and make predictions
        model.fit(X_train, y_train) 
        preds = model.predict_proba(X_cv)[:, 1]

        # compute AUC metric for this CV fold
        fpr, tpr, thresholds = metrics.roc_curve(y_cv, preds)
        roc_auc = metrics.auc(fpr, tpr)
        print "AUC (fold %d/%d): %f" % (i + 1, n, roc_auc)
        mean_auc += roc_auc

    print "Mean AUC: %f" % (mean_auc/n)

    # === Predictions === #
    # When making predictions, retrain the model on the whole training set
    model.fit(X, y)
    preds = model.predict_proba(X_test)[:, 1]
    filename = raw_input("Enter name for submission file: ")
    save_results(preds, filename + ".csv")

if __name__ == '__main__':
    main()

loading data
AUC (fold 1/10): 0.864596
AUC (fold 2/10): 0.872633
AUC (fold 3/10): 0.877062
AUC (fold 4/10): 0.879776
AUC (fold 5/10): 0.849488
AUC (fold 6/10): 0.876831
AUC (fold 7/10): 0.852831
AUC (fold 8/10): 0.859904
AUC (fold 9/10): 0.861278
AUC (fold 10/10): 0.871691
Mean AUC: 0.866609
Enter name for submission file: abc001
